## Save products for each category

In [1]:
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import urllib.request
import sys
import getopt
from multiprocessing.dummy import Pool
import csv
import os
import requests
from sqlalchemy.exc import IntegrityError
import argparse

In [62]:
parent_categry= set(
        ['dresses', 'jeans', 'jackets', 'womens-outerwear', 'womens-pants',
         'shorts', 'skirts', 'sweaters', 'sweatshirts', 'womens-tops'])

In [5]:
#define function to save response
def get_api_json(api_url):
    response = requests.get(api_url).text
    resp_obj = json.loads(response)
    return resp_obj

In [241]:
a=get_api_json('http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+jeans&max-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50')

In [6]:
# define function to parse product info
def parse_product(prod, parent_category):
        pro_id = prod['id']
        name = prod['name']
        image_url = prod['image']['sizes']['Original']['url']
        price=prod['priceLabel']
        try:
            saleprice=prod['salePriceLabel']
        except:
            saleprice=None
        categories = [cat['id'] for cat in prod['categories']]
        return {'id': pro_id,
                'product_name': name,
                'price':price,
                'saleprice':saleprice,
                'image_url': image_url,
                'categories': categories,
                'parent_category': parent_category}

In [7]:
def get_responses(url, offset, product_type):
    api_url = url + '&offset=' + str(offset)
    resp=get_api_json(api_url)
    return resp

In [8]:
def paginate_shopstyle_products(url, offset, product_type):
    api_url = url + '&offset=' + str(offset)
    products = get_api_json(api_url)['products']
    products = [parse_product(
                prod, product_type) for prod in products]
    return products

In [262]:
#scrape
def store_shopstyle_products(product_type):
    product_list=[]
    response_list=[]
    api_url = ''
    if product_type == 'dress':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=dress&max-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'jeans':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+jeans&max-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'jacket':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+jacket&max-price=250&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'outerwear':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+outerwear&max-price=500&r=bloomingdales-us,handm-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us,therealreal-us&sort=Popular&limit=50'
    elif product_type == 'pants':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+pants&max-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'shorts':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+shorts&max-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'skirts':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=skirts&max-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'sweaters':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+sweater&max-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'sweatshirt':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+sweatshirts&max-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'tops':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+tops&max-price=50&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'


        
    offset = 0
    while offset <= 5000:
        print('Getting {} batch {}'.format(product_type, offset))
        product_batch = paginate_shopstyle_products(
            api_url, offset, product_type)
        response=get_responses(api_url, offset, product_type)
        for p in product_batch:
            try:
                product_list.append(p)
            except IntegrityError:
                print('product ', p['id'], ' already exists')
        for r in response:
            response_list.append(response)
        offset += 50
    pickle_name=str(product_type)+'.pickle'
    response_name=str(product_type)+'_response.pickle'
    pickle.dump(product_list, open(pickle_name, 'wb'))
    pickle.dump(response_list, open(response_name, 'wb'))
    return product_list

In [296]:
sweatshirt=store_shopstyle_products('sweatshirt')

Getting sweatshirt batch 0
Getting sweatshirt batch 50
Getting sweatshirt batch 100
Getting sweatshirt batch 150
Getting sweatshirt batch 200
Getting sweatshirt batch 250
Getting sweatshirt batch 300
Getting sweatshirt batch 350
Getting sweatshirt batch 400
Getting sweatshirt batch 450
Getting sweatshirt batch 500
Getting sweatshirt batch 550
Getting sweatshirt batch 600
Getting sweatshirt batch 650
Getting sweatshirt batch 700
Getting sweatshirt batch 750
Getting sweatshirt batch 800
Getting sweatshirt batch 850
Getting sweatshirt batch 900
Getting sweatshirt batch 950
Getting sweatshirt batch 1000
Getting sweatshirt batch 1050
Getting sweatshirt batch 1100
Getting sweatshirt batch 1150
Getting sweatshirt batch 1200
Getting sweatshirt batch 1250
Getting sweatshirt batch 1300
Getting sweatshirt batch 1350
Getting sweatshirt batch 1400
Getting sweatshirt batch 1450
Getting sweatshirt batch 1500
Getting sweatshirt batch 1550
Getting sweatshirt batch 1600
Getting sweatshirt batch 1650
Get

In [301]:
tops=store_shopstyle_products('tops')

Getting tops batch 0
Getting tops batch 50
Getting tops batch 100
Getting tops batch 150
Getting tops batch 200
Getting tops batch 250
Getting tops batch 300
Getting tops batch 350
Getting tops batch 400
Getting tops batch 450
Getting tops batch 500
Getting tops batch 550
Getting tops batch 600
Getting tops batch 650
Getting tops batch 700
Getting tops batch 750
Getting tops batch 800
Getting tops batch 850
Getting tops batch 900
Getting tops batch 950
Getting tops batch 1000
Getting tops batch 1050
Getting tops batch 1100
Getting tops batch 1150
Getting tops batch 1200
Getting tops batch 1250
Getting tops batch 1300
Getting tops batch 1350
Getting tops batch 1400
Getting tops batch 1450
Getting tops batch 1500
Getting tops batch 1550
Getting tops batch 1600
Getting tops batch 1650
Getting tops batch 1700
Getting tops batch 1750
Getting tops batch 1800
Getting tops batch 1850
Getting tops batch 1900
Getting tops batch 1950
Getting tops batch 2000
Getting tops batch 2050
Getting tops ba

In [305]:
pants=store_shopstyle_products('pants')

Getting pants batch 0
Getting pants batch 50
Getting pants batch 100
Getting pants batch 150
Getting pants batch 200
Getting pants batch 250
Getting pants batch 300
Getting pants batch 350
Getting pants batch 400
Getting pants batch 450
Getting pants batch 500
Getting pants batch 550
Getting pants batch 600
Getting pants batch 650
Getting pants batch 700
Getting pants batch 750
Getting pants batch 800
Getting pants batch 850
Getting pants batch 900
Getting pants batch 950
Getting pants batch 1000
Getting pants batch 1050
Getting pants batch 1100
Getting pants batch 1150
Getting pants batch 1200
Getting pants batch 1250
Getting pants batch 1300
Getting pants batch 1350
Getting pants batch 1400
Getting pants batch 1450
Getting pants batch 1500
Getting pants batch 1550
Getting pants batch 1600
Getting pants batch 1650
Getting pants batch 1700
Getting pants batch 1750
Getting pants batch 1800
Getting pants batch 1850
Getting pants batch 1900
Getting pants batch 1950
Getting pants batch 200

In [369]:
dresses=store_shopstyle_products('dress')

Getting dress batch 0
Getting dress batch 50
Getting dress batch 100
Getting dress batch 150
Getting dress batch 200
Getting dress batch 250
Getting dress batch 300
Getting dress batch 350
Getting dress batch 400
Getting dress batch 450
Getting dress batch 500
Getting dress batch 550
Getting dress batch 600
Getting dress batch 650
Getting dress batch 700
Getting dress batch 750
Getting dress batch 800
Getting dress batch 850
Getting dress batch 900
Getting dress batch 950
Getting dress batch 1000
Getting dress batch 1050
Getting dress batch 1100
Getting dress batch 1150
Getting dress batch 1200
Getting dress batch 1250
Getting dress batch 1300
Getting dress batch 1350
Getting dress batch 1400
Getting dress batch 1450
Getting dress batch 1500
Getting dress batch 1550
Getting dress batch 1600
Getting dress batch 1650
Getting dress batch 1700
Getting dress batch 1750
Getting dress batch 1800
Getting dress batch 1850
Getting dress batch 1900
Getting dress batch 1950
Getting dress batch 200

In [374]:
pd.DataFrame(dresses).sort_values(by='id')

,categories,id,image_url,parent_category,price,product_name,saleprice
60,[cocktail-dresses],218767164,https://img.shopstyle-cdn.com/pim/23/0f/230fa7...,dress,$39.98,Xoxo Juniors' Cap-Sleeve Peplum Sheath Dress,$19.99
118,[cocktail-dresses],432120874,https://img.shopstyle-cdn.com/pim/6b/a8/6ba891...,dress,$88,BB Dakota 'Renley' Lace Fit & Flare Dress,None
291,"[petite-dresses, bridesmaid-dresses, evening-d...",433918010,https://img.shopstyle-cdn.com/pim/53/e1/53e181...,dress,$89,Sangria Sleeveless Keyhole Gown,$74.99
1036,[cocktail-dresses],434741392,https://img.shopstyle-cdn.com/pim/9e/76/9e7667...,dress,$89,Karen Kane A-Line Jersey Dress,None
203,"[petite-dresses, day-dresses, cocktail-dresses]",437105639,https://img.shopstyle-cdn.com/pim/f0/43/f04374...,dress,$98,Felicity & Coco Ward Seamed Pencil Dress,None
37,"[petite-dresses, day-dresses]",443526341,https://img.shopstyle-cdn.com/pim/65/f0/65f06d...,dress,$68,LOVEAPPELLA Maxi Dress,None
10,"[petite-dresses, day-dresses]",446985336,https://img.shopstyle-cdn.com/pim/f5/1f/f51fae...,dress,$158,Eliza J Belted Chiffon Maxi Dress,$94.80
337,"[petite-dresses, day-dresses]",453829187,https://img.shopstyle-cdn.com/pim/dd/60/dd6030...,dress,$78,Felicity & Coco Shafter Tie Dye Jersey Halter ...,$46.80
416,[day-dresses],454127444,https://img.shopstyle-cdn.com/pim/ae/c4/aec40a...,dress,$130,Dessy Collection Convertible Wrap Tie Surplice...,$78.98
2,[day-dresses],455310119,https://img.shopstyle-cdn.com/pim/34/a7/34a79f...,dress,$56,Leith Ruched Body-Con Tank Dress,None


In [263]:
jeans=store_shopstyle_products('jeans')

Getting jeans batch 0
Getting jeans batch 50
Getting jeans batch 100
Getting jeans batch 150
Getting jeans batch 200
Getting jeans batch 250
Getting jeans batch 300
Getting jeans batch 350
Getting jeans batch 400
Getting jeans batch 450
Getting jeans batch 500
Getting jeans batch 550
Getting jeans batch 600
Getting jeans batch 650
Getting jeans batch 700
Getting jeans batch 750
Getting jeans batch 800
Getting jeans batch 850
Getting jeans batch 900
Getting jeans batch 950
Getting jeans batch 1000
Getting jeans batch 1050
Getting jeans batch 1100
Getting jeans batch 1150
Getting jeans batch 1200
Getting jeans batch 1250
Getting jeans batch 1300
Getting jeans batch 1350
Getting jeans batch 1400
Getting jeans batch 1450
Getting jeans batch 1500
Getting jeans batch 1550
Getting jeans batch 1600
Getting jeans batch 1650
Getting jeans batch 1700
Getting jeans batch 1750
Getting jeans batch 1800
Getting jeans batch 1850
Getting jeans batch 1900
Getting jeans batch 1950
Getting jeans batch 200

In [370]:
jackets=store_shopstyle_products('jacket')

Getting jacket batch 0
Getting jacket batch 50
Getting jacket batch 100
Getting jacket batch 150
Getting jacket batch 200
Getting jacket batch 250
Getting jacket batch 300
Getting jacket batch 350
Getting jacket batch 400
Getting jacket batch 450
Getting jacket batch 500
Getting jacket batch 550
Getting jacket batch 600
Getting jacket batch 650
Getting jacket batch 700
Getting jacket batch 750
Getting jacket batch 800
Getting jacket batch 850
Getting jacket batch 900
Getting jacket batch 950
Getting jacket batch 1000
Getting jacket batch 1050
Getting jacket batch 1100
Getting jacket batch 1150
Getting jacket batch 1200
Getting jacket batch 1250
Getting jacket batch 1300
Getting jacket batch 1350
Getting jacket batch 1400
Getting jacket batch 1450
Getting jacket batch 1500
Getting jacket batch 1550
Getting jacket batch 1600
Getting jacket batch 1650
Getting jacket batch 1700
Getting jacket batch 1750
Getting jacket batch 1800
Getting jacket batch 1850
Getting jacket batch 1900
Getting j

In [280]:
outerwear=store_shopstyle_products('outerwear')

Getting outerwear batch 0
Getting outerwear batch 50
Getting outerwear batch 100
Getting outerwear batch 150
Getting outerwear batch 200
Getting outerwear batch 250
Getting outerwear batch 300
Getting outerwear batch 350
Getting outerwear batch 400
Getting outerwear batch 450
Getting outerwear batch 500
Getting outerwear batch 550
Getting outerwear batch 600
Getting outerwear batch 650
Getting outerwear batch 700
Getting outerwear batch 750
Getting outerwear batch 800
Getting outerwear batch 850
Getting outerwear batch 900
Getting outerwear batch 950
Getting outerwear batch 1000
Getting outerwear batch 1050
Getting outerwear batch 1100
Getting outerwear batch 1150
Getting outerwear batch 1200
Getting outerwear batch 1250
Getting outerwear batch 1300
Getting outerwear batch 1350
Getting outerwear batch 1400
Getting outerwear batch 1450
Getting outerwear batch 1500
Getting outerwear batch 1550
Getting outerwear batch 1600
Getting outerwear batch 1650
Getting outerwear batch 1700
Getting 

In [284]:
shorts=store_shopstyle_products('shorts')

Getting shorts batch 0
Getting shorts batch 50
Getting shorts batch 100
Getting shorts batch 150
Getting shorts batch 200
Getting shorts batch 250
Getting shorts batch 300
Getting shorts batch 350
Getting shorts batch 400
Getting shorts batch 450
Getting shorts batch 500
Getting shorts batch 550
Getting shorts batch 600
Getting shorts batch 650
Getting shorts batch 700
Getting shorts batch 750
Getting shorts batch 800
Getting shorts batch 850
Getting shorts batch 900
Getting shorts batch 950
Getting shorts batch 1000
Getting shorts batch 1050
Getting shorts batch 1100
Getting shorts batch 1150
Getting shorts batch 1200
Getting shorts batch 1250
Getting shorts batch 1300
Getting shorts batch 1350
Getting shorts batch 1400
Getting shorts batch 1450
Getting shorts batch 1500
Getting shorts batch 1550
Getting shorts batch 1600
Getting shorts batch 1650
Getting shorts batch 1700
Getting shorts batch 1750
Getting shorts batch 1800
Getting shorts batch 1850
Getting shorts batch 1900
Getting s

In [289]:
skirts=store_shopstyle_products('skirts')

Getting skirts batch 0
Getting skirts batch 50
Getting skirts batch 100
Getting skirts batch 150
Getting skirts batch 200
Getting skirts batch 250
Getting skirts batch 300
Getting skirts batch 350
Getting skirts batch 400
Getting skirts batch 450
Getting skirts batch 500
Getting skirts batch 550
Getting skirts batch 600
Getting skirts batch 650
Getting skirts batch 700
Getting skirts batch 750
Getting skirts batch 800
Getting skirts batch 850
Getting skirts batch 900
Getting skirts batch 950
Getting skirts batch 1000
Getting skirts batch 1050
Getting skirts batch 1100
Getting skirts batch 1150
Getting skirts batch 1200
Getting skirts batch 1250
Getting skirts batch 1300
Getting skirts batch 1350
Getting skirts batch 1400
Getting skirts batch 1450
Getting skirts batch 1500
Getting skirts batch 1550
Getting skirts batch 1600
Getting skirts batch 1650
Getting skirts batch 1700
Getting skirts batch 1750
Getting skirts batch 1800
Getting skirts batch 1850
Getting skirts batch 1900
Getting s

In [293]:
sweaters=store_shopstyle_products('sweaters')

Getting sweaters batch 0
Getting sweaters batch 50
Getting sweaters batch 100
Getting sweaters batch 150
Getting sweaters batch 200
Getting sweaters batch 250
Getting sweaters batch 300
Getting sweaters batch 350
Getting sweaters batch 400
Getting sweaters batch 450
Getting sweaters batch 500
Getting sweaters batch 550
Getting sweaters batch 600
Getting sweaters batch 650
Getting sweaters batch 700
Getting sweaters batch 750
Getting sweaters batch 800
Getting sweaters batch 850
Getting sweaters batch 900
Getting sweaters batch 950
Getting sweaters batch 1000
Getting sweaters batch 1050
Getting sweaters batch 1100
Getting sweaters batch 1150
Getting sweaters batch 1200
Getting sweaters batch 1250
Getting sweaters batch 1300
Getting sweaters batch 1350
Getting sweaters batch 1400
Getting sweaters batch 1450
Getting sweaters batch 1500
Getting sweaters batch 1550
Getting sweaters batch 1600
Getting sweaters batch 1650
Getting sweaters batch 1700
Getting sweaters batch 1750
Getting sweater

In [188]:
with open('dress.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [306]:
pd.DataFrame(pants)

,categories,id,image_url,parent_category,price,product_name,saleprice
0,[womens-pants],733565937,https://img.shopstyle-cdn.com/pim/27/e9/27e9cf...,pants,$59,Socialite Popover Jumpsuit,None
1,"[petite-pants, womens-pants]",467736264,https://img.shopstyle-cdn.com/pim/5d/0f/5d0fc5...,pants,$88,LOVEAPPELLA Short Sleeve Wrap Top Jumpsuit,None
2,[womens-pants],689154906,https://img.shopstyle-cdn.com/pim/85/05/8505f1...,pants,$178,Eileen Fisher Stretch Crepe Back-Zip Pants,$89
3,"[casual-pants, cropped-pants, mens-casual-pants]",733072335,https://img.shopstyle-cdn.com/pim/87/b8/87b886...,pants,$90,XCVI Kahiwa Ruched Cropped Pants,$35.97
4,[womens-pants],732427136,https://img.shopstyle-cdn.com/pim/87/31/87316c...,pants,$79,Leith Surplice Jumpsuit,$39.49
5,[womens-pants],724256882,https://img.shopstyle-cdn.com/pim/3a/e4/3ae410...,pants,$109,Adelyn Rae Landra Wrap Front Cotton & Linen Cu...,$65.40
6,[womens-pants],732435946,https://img.shopstyle-cdn.com/pim/bc/24/bc24ea...,pants,$69,Leith Tie Front Pant,$34.50
7,[cropped-pants],670575285,https://img.shopstyle-cdn.com/pim/7f/05/7f05d3...,pants,$75,Leith Crop Wide Leg Jumpsuit,$44.98
8,[womens-pants],699777603,https://img.shopstyle-cdn.com/pim/3d/cf/3dcf55...,pants,$168,Eileen Fisher Stretch Crepe Boot-Cut Pants,$75
9,[dress-pants],445218330,https://img.shopstyle-cdn.com/pim/27/d1/27d127...,pants,$98,Vince Camuto Faux Wrap Jersey Jumpsuit,None


In [367]:
#save images
def get_img(product):
    path = './data/test/tops/'+str(product['id'])+'.jpg'
    url = product['image_url']
    print('Getting ', str(product['id']), ': ', path, ', ', url)
    urllib.request.urlretrieve(url, path)
    
def save_shopstyle_products(product_type):
    for i in product_type:
        try:
            get_img(i)
        except:
            print('already exist')

In [299]:
df=pd.DataFrame(sweatshirt)

In [278]:
df.loc[df['id']==739511288]

,categories,id,image_url,parent_category,price,product_name,saleprice
2877,[casual-jackets],739511288,https://img.shopstyle-cdn.com/pim/c2/06/c2065c...,jacket,$168,Free People Lauren Band Jacket,$100.80


In [368]:
save_shopstyle_products(tops_test)

Getting  692156409 :  ./data/test/tops/692156409.jpg ,  https://img.shopstyle-cdn.com/pim/73/dc/73dc2d90e905e67db8a9eb017a343f6b_best.jpg
Getting  693359518 :  ./data/test/tops/693359518.jpg ,  https://img.shopstyle-cdn.com/pim/26/a4/26a4367f4aa7bd3552fe10a3bfc4215e_best.jpg
Getting  673521436 :  ./data/test/tops/673521436.jpg ,  https://img.shopstyle-cdn.com/pim/ae/14/ae1480b9708527d3ff5e189efbb1f0e8_best.jpg
Getting  651716636 :  ./data/test/tops/651716636.jpg ,  https://img.shopstyle-cdn.com/pim/4a/9e/4a9e047e8e7f3c2eb594eab1675d31b4_best.jpg
Getting  734154784 :  ./data/test/tops/734154784.jpg ,  https://img.shopstyle-cdn.com/pim/04/93/0493536893de8a3f1e1a8d1c846ce3f3_best.jpg
Getting  656248343 :  ./data/test/tops/656248343.jpg ,  https://img.shopstyle-cdn.com/pim/10/25/10250219ed69836103afe411dfcc346a_best.jpg
Getting  435423474 :  ./data/test/tops/435423474.jpg ,  https://img.shopstyle-cdn.com/pim/d9/4c/d94c4d24adf37de8b9fa3715f2686932_best.jpg
Getting  544695895 :  ./data/test/

Getting  737579022 :  ./data/test/tops/737579022.jpg ,  https://img.shopstyle-cdn.com/pim/ab/b2/abb29dc7ab4ba02c4baa70435c53eba2_best.jpg
Getting  712379768 :  ./data/test/tops/712379768.jpg ,  https://img.shopstyle-cdn.com/pim/de/99/de99fe82fe4347d50c3614ab8a902cae_best.jpg
Getting  733712254 :  ./data/test/tops/733712254.jpg ,  https://img.shopstyle-cdn.com/pim/b3/72/b3722351f3ded71766ad4b4d885cd588_best.jpg
Getting  434055302 :  ./data/test/tops/434055302.jpg ,  https://img.shopstyle-cdn.com/pim/25/f1/25f13162331c1bd5d17962ed98bab477_best.jpg
Getting  717836288 :  ./data/test/tops/717836288.jpg ,  https://img.shopstyle-cdn.com/pim/fc/96/fc96e31918200581d83437812cde643a_best.jpg
Getting  739427492 :  ./data/test/tops/739427492.jpg ,  https://img.shopstyle-cdn.com/pim/40/60/40604d4a26c8f355e380052880263bd4_best.jpg
Getting  692843252 :  ./data/test/tops/692843252.jpg ,  https://img.shopstyle-cdn.com/pim/ca/f6/caf660b7b9ea7c1170f3cd89954c3c72_best.jpg
Getting  713797042 :  ./data/test/

Getting  732426562 :  ./data/test/tops/732426562.jpg ,  https://img.shopstyle-cdn.com/pim/32/3b/323b8772395ffb7d0798e2a25528b825_best.jpg
Getting  707259083 :  ./data/test/tops/707259083.jpg ,  https://img.shopstyle-cdn.com/pim/d1/94/d1944f3f3b98061152e83f63e243d531_best.jpg
Getting  702085428 :  ./data/test/tops/702085428.jpg ,  https://img.shopstyle-cdn.com/pim/86/39/8639f5cb36a4009ec803b713e8fe3093_best.jpg
Getting  665720324 :  ./data/test/tops/665720324.jpg ,  https://img.shopstyle-cdn.com/pim/23/a2/23a2b8a384ada164b7a867aa67793e42_best.jpg
Getting  741776375 :  ./data/test/tops/741776375.jpg ,  https://img.shopstyle-cdn.com/pim/fd/68/fd688ba51f39f4019f3f64c4023d2da3_best.jpg
Getting  666390597 :  ./data/test/tops/666390597.jpg ,  https://img.shopstyle-cdn.com/pim/b5/d5/b5d51553ce850c580df8821dc8d3bdd7_best.jpg
Getting  741942423 :  ./data/test/tops/741942423.jpg ,  https://img.shopstyle-cdn.com/pim/48/9b/489b4a0ab96dc51f0b902002f8f10ffa_best.jpg
Getting  733577822 :  ./data/test/

Getting  652730717 :  ./data/test/tops/652730717.jpg ,  https://img.shopstyle-cdn.com/pim/f5/ce/f5cedecb40108b7b900b1eb54afd822c_best.jpg
Getting  735672063 :  ./data/test/tops/735672063.jpg ,  https://img.shopstyle-cdn.com/pim/c9/a2/c9a2a0b67318b6aed18fae1ee2e3d33e_best.jpg
Getting  721381403 :  ./data/test/tops/721381403.jpg ,  https://img.shopstyle-cdn.com/pim/3a/a2/3aa24c41c2df0a282dd272f3bb289200_best.jpg
Getting  668437452 :  ./data/test/tops/668437452.jpg ,  https://img.shopstyle-cdn.com/pim/c8/34/c834ffd9032fd9f4fa3a0657ad188fe7_best.jpg
Getting  712127561 :  ./data/test/tops/712127561.jpg ,  https://img.shopstyle-cdn.com/pim/a3/40/a340f7c07865b7d4e0d5782c25c630b8_best.jpg
Getting  738323491 :  ./data/test/tops/738323491.jpg ,  https://img.shopstyle-cdn.com/pim/44/75/4475b554f8e90bb144b197da2a71911a_best.jpg
Getting  736767716 :  ./data/test/tops/736767716.jpg ,  https://img.shopstyle-cdn.com/pim/7c/58/7c5815a5143df512ffde433bf9871165_best.jpg
Getting  738178779 :  ./data/test/

Getting  731123110 :  ./data/test/tops/731123110.jpg ,  https://img.shopstyle-cdn.com/pim/55/a6/55a6c29f96b07a805a024c2641de90c8_best.jpg
Getting  729603732 :  ./data/test/tops/729603732.jpg ,  https://img.shopstyle-cdn.com/pim/0d/7d/0d7da0fd7804912148550ea1f65c1b64_best.jpg
Getting  726963169 :  ./data/test/tops/726963169.jpg ,  https://img.shopstyle-cdn.com/pim/2e/85/2e859050f784769749fe9bf49b834367_best.jpg
Getting  737326106 :  ./data/test/tops/737326106.jpg ,  https://img.shopstyle-cdn.com/pim/52/4c/524ca59143a667cfdb0d7f8706b905bd_best.jpg
Getting  726571758 :  ./data/test/tops/726571758.jpg ,  https://img.shopstyle-cdn.com/pim/d9/95/d9958e45d491a217b0b2950eb43b211c_best.jpg
Getting  733087488 :  ./data/test/tops/733087488.jpg ,  https://img.shopstyle-cdn.com/pim/20/32/2032d63c5f027d11cfd7e80438bdfeae_best.jpg
Getting  730326059 :  ./data/test/tops/730326059.jpg ,  https://img.shopstyle-cdn.com/pim/53/3f/533f0dd16e4cc4811fa9879832528262_best.jpg
Getting  725126308 :  ./data/test/

...

Getting  670273890 :  ./data/test/tops/670273890.jpg ,  https://img.shopstyle-cdn.com/pim/12/05/1205f0370ae0aeba5cf97e9c9d639bab_best.jpg
Getting  489343547 :  ./data/test/tops/489343547.jpg ,  https://img.shopstyle-cdn.com/pim/1d/76/1d76f76cb34ae915b16a74d750147f85_best.jpg
Getting  712621914 :  ./data/test/tops/712621914.jpg ,  https://img.shopstyle-cdn.com/pim/b8/67/b867c311516c1bef8246a47512bf9ec2_best.jpg
Getting  734185098 :  ./data/test/tops/734185098.jpg ,  https://img.shopstyle-cdn.com/pim/17/22/172224e7b1c1a39a96671ec625e071b9_best.jpg
Getting  740565974 :  ./data/test/tops/740565974.jpg ,  https://img.shopstyle-cdn.com/pim/93/30/93300b1ae49f2a77a5e502b4f150b96d_best.jpg
Getting  708256028 :  ./data/test/tops/708256028.jpg ,  https://img.shopstyle-cdn.com/pim/0d/f7/0df7f5160da1d1e8df2c5d2bb72b2aff_best.jpg
Getting  608838370 :  ./data/test/tops/608838370.jpg ,  https://img.shopstyle-cdn.com/pim/24/22/24222684c56a99c3c47367c32907fdb7_best.jpg
Getting  741232534 :  ./data/test/

Getting  740746900 :  ./data/test/tops/740746900.jpg ,  https://img.shopstyle-cdn.com/pim/70/c4/70c4de85f2210c45e6e3ea385aa0071b_best.jpg
Getting  741098512 :  ./data/test/tops/741098512.jpg ,  https://img.shopstyle-cdn.com/pim/2f/ab/2fab3a43c87d8197bc545a483ffed273_best.jpg
Getting  740433618 :  ./data/test/tops/740433618.jpg ,  https://img.shopstyle-cdn.com/pim/e0/a3/e0a3db55f96b5435b37b794b5b54e141_best.jpg
Getting  739460344 :  ./data/test/tops/739460344.jpg ,  https://img.shopstyle-cdn.com/pim/e0/a4/e0a4ff7783cead3929b14656d685a3ee_best.jpg
Getting  739826582 :  ./data/test/tops/739826582.jpg ,  https://img.shopstyle-cdn.com/pim/88/01/88013dbd781b43bc92d8f50dc72f9879_best.jpg
Getting  738702834 :  ./data/test/tops/738702834.jpg ,  https://img.shopstyle-cdn.com/pim/d8/52/d852a902b8cf746dcea6c81afab9ae3b_best.jpg
Getting  740443677 :  ./data/test/tops/740443677.jpg ,  https://img.shopstyle-cdn.com/pim/93/3f/933f26f1be6e1472d198ee2e2df58448_best.jpg
Getting  732946873 :  ./data/test/

Getting  736061088 :  ./data/test/tops/736061088.jpg ,  https://img.shopstyle-cdn.com/pim/0f/7f/0f7f7a76a9778e0e5ffddf88e33e026c_best.jpg
Getting  735849086 :  ./data/test/tops/735849086.jpg ,  https://img.shopstyle-cdn.com/pim/e0/38/e038a411b3169f718d99f491c2bc9370_best.jpg
Getting  732942746 :  ./data/test/tops/732942746.jpg ,  https://img.shopstyle-cdn.com/pim/86/2a/862ad9935fdcc24257ef6109918fb86a_best.jpg
Getting  732704012 :  ./data/test/tops/732704012.jpg ,  https://img.shopstyle-cdn.com/pim/3b/df/3bdf62f84106e74a14b8c959c8af1476_best.jpg
Getting  730188283 :  ./data/test/tops/730188283.jpg ,  https://img.shopstyle-cdn.com/pim/8a/26/8a26e780312b2d4c7a95016616345273_best.jpg
Getting  729016270 :  ./data/test/tops/729016270.jpg ,  https://img.shopstyle-cdn.com/pim/2d/cf/2dcf7fb304fa3b738cd9ed2c66d3c0b1_best.jpg
Getting  729401471 :  ./data/test/tops/729401471.jpg ,  https://img.shopstyle-cdn.com/pim/3b/e8/3be84d97f7806bdd9c909c0446cb5460_best.jpg
Getting  721670410 :  ./data/test/

Getting  726465446 :  ./data/test/tops/726465446.jpg ,  https://img.shopstyle-cdn.com/pim/24/1b/241ba4792cb77232096e1b137911a230_best.jpg
Getting  721475170 :  ./data/test/tops/721475170.jpg ,  https://img.shopstyle-cdn.com/pim/0d/ff/0dff07729994db54d232b67a4c4a231a_best.jpg
Getting  738806855 :  ./data/test/tops/738806855.jpg ,  https://img.shopstyle-cdn.com/pim/62/97/6297178ce94546ece93b4391f6b8d207_best.jpg
Getting  726016681 :  ./data/test/tops/726016681.jpg ,  https://img.shopstyle-cdn.com/pim/a6/00/a600670e665530e3a560a079f551805a_best.jpg
Getting  627833637 :  ./data/test/tops/627833637.jpg ,  https://img.shopstyle-cdn.com/pim/0b/e1/0be16423c71c1b23cdc5677780e81cde_best.jpg
Getting  731456147 :  ./data/test/tops/731456147.jpg ,  https://img.shopstyle-cdn.com/pim/e0/d5/e0d5bd704cbe02e0db1ee7136b231408_best.jpg
Getting  706065854 :  ./data/test/tops/706065854.jpg ,  https://img.shopstyle-cdn.com/pim/c0/5a/c05a7fd9aa947820079385ca43878295_best.jpg
Getting  718633900 :  ./data/test/

Getting  730442140 :  ./data/test/tops/730442140.jpg ,  https://img.shopstyle-cdn.com/pim/58/ca/58cac267c5dc6894282dc1334d23fd8e_best.jpg
Getting  729508422 :  ./data/test/tops/729508422.jpg ,  https://img.shopstyle-cdn.com/pim/c3/8b/c38b2b4179227d00eb8ac7802de08ec9_best.jpg
Getting  728685935 :  ./data/test/tops/728685935.jpg ,  https://img.shopstyle-cdn.com/pim/45/41/454183eca829ef84c61f4eb4dc38bf0e_best.jpg
Getting  725275091 :  ./data/test/tops/725275091.jpg ,  https://img.shopstyle-cdn.com/pim/ea/2b/ea2bced27a4bd4abcf4d329d03b9685d_best.jpg
Getting  715078699 :  ./data/test/tops/715078699.jpg ,  https://img.shopstyle-cdn.com/pim/c7/2c/c72cdef06f464bb3de91729d490ac049_best.jpg
Getting  715078622 :  ./data/test/tops/715078622.jpg ,  https://img.shopstyle-cdn.com/pim/b2/2c/b22c0e016f7f1739a278470991119a9e_best.jpg
Getting  733869861 :  ./data/test/tops/733869861.jpg ,  https://img.shopstyle-cdn.com/pim/ec/82/ec82869f215ae3afc56ccd388da8fee5_best.jpg
Getting  732000054 :  ./data/test/

In [ ]:
urllib.request.urlretrieve(url, path)

In [200]:
df_tops=pd.DataFrame(tops)

In [206]:
df_tops['id']

0       472140360
1       649091558
2       668894662
3       666446889
4       718360833
5       726920058
6       689991773
7       655435014
8       738877251
9       732704078
10      736059773
11      499624871
12      640690911
13      732439024
14      693199506
15      738529808
16      732438160
17      715251142
18      628733768
19      625727418
20      632777593
21      739428023
22      432874704
23      732434843
24      714479280
25      718991160
26      735673893
27      733273680
28      652757603
29      543854299
          ...    
4070    714992303
4071    738985541
4072    737720208
4073    653910002
4074    737719552
4075    733260817
4076    737985515
4077    739863936
4078    735674673
4079    733186939
4080    718914728
4081    723484502
4082    516604175
4083    725001157
4084    739785814
4085    706269637
4086    739775395
4087    738529882
4088    735690947
4089    709680792
4090    719012496
4091    737986250
4092    719469712
4093    703095998
4094    73

In [153]:
data

[{'categories': ['dresses'],
  'id': 605820765,
  'image_url': 'https://img.shopstyle-cdn.com/pim/37/08/37087e9492b2eaa1407d99e3760afc91_best.jpg',
  'parent_category': 'dress',
  'price': '$548',
  'product_name': 'Short Sleeve Lace Gown'},
 {'categories': ['day-dresses'],
  'id': 642632776,
  'image_url': 'https://img.shopstyle-cdn.com/pim/cc/00/cc0081e81c98124fd6a9f738402cf778_best.jpg',
  'parent_category': 'dress',
  'price': '$128',
  'product_name': 'Eliza J Belted Fit & Flare Dress'},
 {'categories': ['cocktail-dresses'],
  'id': 607750567,
  'image_url': 'https://img.shopstyle-cdn.com/pim/7f/e4/7fe4a76d1c44ebcea5647be22f693856_best.jpg',
  'parent_category': 'dress',
  'price': '$238',
  'product_name': 'Privacy Please Krause Dress'},
 {'categories': ['evening-dresses'],
  'id': 638182592,
  'image_url': 'https://img.shopstyle-cdn.com/pim/9b/86/9b865ee07874c5a023d60c2fb6f79cb8_best.jpg',
  'parent_category': 'dress',
  'price': '$998',
  'product_name': 'Tadashi Shoji Long Sle

In [160]:
from keras.applications import VGG16
import pandas as pd
import numpy as np

from keras.optimizers import SGD

model = VGG16()
!ls -lha ~/.keras/models
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


553467904/553467096 [==============================] - 359s 1us/step
total 1089536
drwxr-xr-x  3 Minmin  staff    96B Aug 29 16:15 .
drwxr-xr-x  5 Minmin  staff   160B Aug 29 16:15 ..
-rw-r--r--  1 Minmin  staff   528M Aug 29 16:21 vgg16_weights_tf_dim_ordering_tf_kernels.h5


In [161]:
from skimage.io import imread
from skimage.transform import resize
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

def get_input(img_path):
    img = imread(img_path)
    img = resize(img, (224, 224), preserve_range=True).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def get_predictions(img_path, decode=True):
    input_ = get_input(img_path)
    out = model.predict(input_)
    if decode:
        return decode_predictions(out,top=10)
    else:
        return out

In [172]:
img = resize(imread('dresses/642632776.jpg'), (224, 224), PIL.Image.ANTIALIAS)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [175]:
import PIL
from PIL import Image
img = Image.open('dresses/642632776.jpg')
img = img.resize((224, 224), PIL.Image.ANTIALIAS)
img.save('resized_image.jpg')

This is a dress
![](./dresses/698465217.jpg)

In [187]:
img_file = '/Users/Minmin/ds/shopstyle/dresses/698465217.jpg'
get_predictions(img_file)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[('n03450230', 'gown', 0.541102),
  ('n03534580', 'hoopskirt', 0.16091764),
  ('n03770439', 'miniskirt', 0.08021285),
  ('n03710637', 'maillot', 0.051840894),
  ('n03866082', 'overskirt', 0.03333488),
  ('n03877472', 'pajama', 0.0320876),
  ('n03710721', 'maillot', 0.027976468),
  ('n04136333', 'sarong', 0.0149462465),
  ('n04525038', 'velvet', 0.006054087),
  ('n03832673', 'notebook', 0.0040826914)]]

In [116]:
a=get_api_json('http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+pants&sort=Popular&offset=51&limit=50')

In [127]:
pickle.dump(a, open('dress.pickle', 'wb'))


In [103]:
a['products'][0]['categories']

[{'fullName': "Women's Wide Leg Pants",
  'id': 'wide-leg-pants',
  'localizedId': 'wide-leg-pants',
  'name': 'Wide Leg Pants',
  'numId': 91,
  'shortName': 'Wide Leg'}]

In [31]:
a['products'][0]['image']['sizes']['Original']['url']

'https://img.shopstyle-cdn.com/pim/68/4a/684a944567dac66a844edf4f2221756d_best.jpg'

In [41]:
b=get_api_json('http://api.shopstyle.com/api/v2/categories?pid=shopstyle')

In [61]:
import json
import pandas as pd
from pandas.io.json import json_normalize
json_normalize(b['categories'])['fullName']

0                   Women's Fashion
1                   Women's Clothes
2          Women's Athletic Clothes
3          Women's Athletic Jackets
4            Women's Athletic Pants
5           Women's Athletic Shorts
6           Women's Athletic Skirts
7           Sports Bras & Underwear
8             Women's Athletic Tops
9                            Bridal
10                            Bride
11                     Bridal Gowns
12        Bridal Veils & Headpieces
13                      Bridal Bags
14                   Bridal Jewelry
15                  Bridal Lingerie
16                     Bridal Shoes
17                       Bridesmaid
18              Bridesmaid' Dresses
19                 Bridesmaid' Bags
20              Bridesmaid' Jewelry
21              Mother of the Bride
22     Mother of the Bride' Dresses
23        Mother of the Bride' Bags
24     Mother of the Bride' Jewelry
25                    Women's Jeans
26            Women's Bootcut Jeans
27            Women's Classi

In [ ]:
def get_api_json(api_url):
    response = requests.get(api_url).text
    resp_obj = json.loads(response)
    return resp_obj

In [229]:
response = requests.get('http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=dressmax-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50')

In [230]:
response.status_code

200

In [231]:
print(response.text)

{"metadata":{"offset":0,"limit":50,"total":0,"category":{"numId":1,"id":"clothes-shoes-and-jewelry","name":"Clothes and Shoes","shortName":"All Categories","fullName":"Clothes, Shoes, & Jewelry","localizedIds":{"en-US":"clothes-shoes-and-jewelry","en-CA":"clothes-shoes-and-jewelry","en-AU":"clothes-shoes-and-jewelry","en-GB":"clothes-shoes-and-jewelry","ja-JP":"ファッション-シューズ-アクセサリー"},"localizedId":"clothes-shoes-and-jewelry"},"majorCategory":{"numId":1,"id":"clothes-shoes-and-jewelry","name":"Clothes and Shoes","shortName":"All Categories","fullName":"Clothes, Shoes, & Jewelry","localizedId":"clothes-shoes-and-jewelry"},"showSizeFilter":false,"showColorFilter":true,"showHeelHeightFilter":false,"showConditionFilter":false,"filters":[{"id":"750","name":"Last Call by Neiman Marcus","type":"Retailer","urlIdentifier":"last-call-by-neiman-marcus-us"},{"id":"1","name":"Nordstrom","type":"Retailer","urlIdentifier":"nordstrom-us"},{"id":"2","name":"Macy's","type":"Retailer","urlIdentifier":"macys

In [2]:
def store_shopstyle_products_test(product_type):
    product_list=[]
    response_list=[]
    api_url = ''
    if product_type == 'dress':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=dress&min-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'jeans':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+jeans&min-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'jacket':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+jacket&min-price=250&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'outerwear':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+outerwear&min-price=500&r=bloomingdales-us,handm-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us,therealreal-us&sort=Popular&limit=50'
    elif product_type == 'pants':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+pants&min-price=80&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'
    elif product_type == 'shorts':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+shorts&min-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'skirts':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=skirts&min-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'sweaters':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+sweater&min-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'sweatshirt':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+sweatshirts&min-price=80&r=bloomingdales-us,jcpenney-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,mango-us,neiman-marcus-us,nordstrom-rack-us,nordstrom-us,off-5th-us,rue-la-la-us&sort=Popular&limit=50'
    elif product_type == 'tops':
        api_url = 'http://api.shopstyle.com/api/v2/products?pid=shopstyle&fts=women+tops&min-price=50&r=bloomingdales-us,last-call-by-neiman-marcus-us,lord-and-taylor-us,macys-us,nordstrom-rack-us,nordstrom-us&sort=Popular&limit=50'


        
    offset = 0
    while offset <= 950:
        print('Getting {} batch {}'.format(product_type, offset))
        product_batch = paginate_shopstyle_products(
            api_url, offset, product_type)
        response=get_responses(api_url, offset, product_type)
        for p in product_batch:
            try:
                product_list.append(p)
            except IntegrityError:
                print('product ', p['id'], ' already exists')
        for r in response:
            response_list.append(response)
        offset += 50
    pickle_name=str(product_type)+'_test.pickle'
    response_name=str(product_type)+'_response_test.pickle'
    pickle.dump(product_list, open(pickle_name, 'wb'))
    pickle.dump(response_list, open(response_name, 'wb'))
    return product_list

In [337]:
jeans_test=store_shopstyle_products_test('jeans')

Getting jeans batch 0
Getting jeans batch 50
Getting jeans batch 100
Getting jeans batch 150
Getting jeans batch 200
Getting jeans batch 250
Getting jeans batch 300
Getting jeans batch 350
Getting jeans batch 400
Getting jeans batch 450
Getting jeans batch 500
Getting jeans batch 550
Getting jeans batch 600
Getting jeans batch 650
Getting jeans batch 700
Getting jeans batch 750
Getting jeans batch 800
Getting jeans batch 850
Getting jeans batch 900
Getting jeans batch 950


In [344]:
outerwear_test=store_shopstyle_products_test('outerwear')

Getting outerwear batch 0
Getting outerwear batch 50
Getting outerwear batch 100
Getting outerwear batch 150
Getting outerwear batch 200
Getting outerwear batch 250
Getting outerwear batch 300
Getting outerwear batch 350
Getting outerwear batch 400
Getting outerwear batch 450
Getting outerwear batch 500
Getting outerwear batch 550
Getting outerwear batch 600
Getting outerwear batch 650
Getting outerwear batch 700
Getting outerwear batch 750
Getting outerwear batch 800
Getting outerwear batch 850
Getting outerwear batch 900
Getting outerwear batch 950


In [354]:
len(shorts_test)

1000

In [349]:
pants_test=store_shopstyle_products_test('pants')

Getting pants batch 0
Getting pants batch 50
Getting pants batch 100
Getting pants batch 150
Getting pants batch 200
Getting pants batch 250
Getting pants batch 300
Getting pants batch 350
Getting pants batch 400
Getting pants batch 450
Getting pants batch 500
Getting pants batch 550
Getting pants batch 600
Getting pants batch 650
Getting pants batch 700
Getting pants batch 750
Getting pants batch 800
Getting pants batch 850
Getting pants batch 900
Getting pants batch 950


In [353]:
shorts_test=store_shopstyle_products_test('shorts')

Getting shorts batch 0
Getting shorts batch 50
Getting shorts batch 100
Getting shorts batch 150
Getting shorts batch 200
Getting shorts batch 250
Getting shorts batch 300
Getting shorts batch 350
Getting shorts batch 400
Getting shorts batch 450
Getting shorts batch 500
Getting shorts batch 550
Getting shorts batch 600
Getting shorts batch 650
Getting shorts batch 700
Getting shorts batch 750
Getting shorts batch 800
Getting shorts batch 850
Getting shorts batch 900
Getting shorts batch 950


In [357]:
skirts_test=store_shopstyle_products_test('skirts')

Getting skirts batch 0
Getting skirts batch 50
Getting skirts batch 100
Getting skirts batch 150
Getting skirts batch 200
Getting skirts batch 250
Getting skirts batch 300
Getting skirts batch 350
Getting skirts batch 400
Getting skirts batch 450
Getting skirts batch 500
Getting skirts batch 550
Getting skirts batch 600
Getting skirts batch 650
Getting skirts batch 700
Getting skirts batch 750
Getting skirts batch 800
Getting skirts batch 850
Getting skirts batch 900
Getting skirts batch 950


In [360]:
sweaters_test=store_shopstyle_products_test('sweaters')

Getting sweaters batch 0
Getting sweaters batch 50
Getting sweaters batch 100
Getting sweaters batch 150
Getting sweaters batch 200
Getting sweaters batch 250
Getting sweaters batch 300
Getting sweaters batch 350
Getting sweaters batch 400
Getting sweaters batch 450
Getting sweaters batch 500
Getting sweaters batch 550
Getting sweaters batch 600
Getting sweaters batch 650
Getting sweaters batch 700
Getting sweaters batch 750
Getting sweaters batch 800
Getting sweaters batch 850
Getting sweaters batch 900
Getting sweaters batch 950


In [363]:
sweatshirt_test=store_shopstyle_products_test('sweatshirt')

Getting sweatshirt batch 0
Getting sweatshirt batch 50
Getting sweatshirt batch 100
Getting sweatshirt batch 150
Getting sweatshirt batch 200
Getting sweatshirt batch 250
Getting sweatshirt batch 300
Getting sweatshirt batch 350
Getting sweatshirt batch 400
Getting sweatshirt batch 450
Getting sweatshirt batch 500
Getting sweatshirt batch 550
Getting sweatshirt batch 600
Getting sweatshirt batch 650
Getting sweatshirt batch 700
Getting sweatshirt batch 750
Getting sweatshirt batch 800
Getting sweatshirt batch 850
Getting sweatshirt batch 900
Getting sweatshirt batch 950


In [366]:
tops_test=store_shopstyle_products_test('tops')

Getting tops batch 0
Getting tops batch 50
Getting tops batch 100
Getting tops batch 150
Getting tops batch 200
Getting tops batch 250
Getting tops batch 300
Getting tops batch 350
Getting tops batch 400
Getting tops batch 450
Getting tops batch 500
Getting tops batch 550
Getting tops batch 600
Getting tops batch 650
Getting tops batch 700
Getting tops batch 750
Getting tops batch 800
Getting tops batch 850
Getting tops batch 900
Getting tops batch 950


In [10]:
import pickle
store_shopstyle_products_test('dress')

Getting dress batch 0
Getting dress batch 50
Getting dress batch 100
Getting dress batch 150
Getting dress batch 200
Getting dress batch 250
Getting dress batch 300
Getting dress batch 350
Getting dress batch 400
Getting dress batch 450
Getting dress batch 500
Getting dress batch 550
Getting dress batch 600
Getting dress batch 650
Getting dress batch 700
Getting dress batch 750
Getting dress batch 800
Getting dress batch 850
Getting dress batch 900
Getting dress batch 950


[{'categories': ['day-dresses'],
  'id': 642632776,
  'image_url': 'https://img.shopstyle-cdn.com/pim/cc/00/cc0081e81c98124fd6a9f738402cf778_best.jpg',
  'parent_category': 'dress',
  'price': '$128',
  'product_name': 'Eliza J Belted Fit & Flare Dress',
  'saleprice': '$76.80'},
 {'categories': ['evening-dresses'],
  'id': 638182592,
  'image_url': 'https://img.shopstyle-cdn.com/pim/9b/86/9b865ee07874c5a023d60c2fb6f79cb8_best.jpg',
  'parent_category': 'dress',
  'price': '$998',
  'product_name': 'Tadashi Shoji Long Sleeve A-Line Sheath Gown',
  'saleprice': '$853.14'},
 {'categories': ['evening-dresses'],
  'id': 715586211,
  'image_url': 'https://img.shopstyle-cdn.com/pim/e4/0a/e40a819601b7eeb15fbe48b76fa4ab24_best.jpg',
  'parent_category': 'dress',
  'price': '$179',
  'product_name': 'Adrianna Papell Embellished Bodice Gown',
  'saleprice': '$134.25'},
 {'categories': ['evening-dresses'],
  'id': 733703367,
  'image_url': 'https://img.shopstyle-cdn.com/pim/22/8f/228fdf06ecb1a4f6

In [157]:
train_dress_list=!ls data/train/sweatshirt
train_dress_list=[int(i.split('.')[0]) for i in train_dress_list]

In [158]:
train_dress=pd.read_pickle('metadata/train_meta/sweatshirt.pickle')

In [159]:
train_dress=list(pd.DataFrame(train_dress)['id'])

In [160]:
need_scrape=[i for i in train_dress_list if i not in train_dress]

In [161]:
len(need_scrape)

0

In [130]:
need_scrape_info=[]
for i in need_scrape:
    print(i)
    url='http://api.shopstyle.com/api/v2/products/'+str(i)+'?pid=shopstyle'
    prod=get_api_json(url)
    pro_id = prod['id']
    name = prod['name']
    image_url = prod['image']['sizes']['Original']['url']
    price=prod['priceLabel']
    try:
        saleprice=prod['salePriceLabel']
    except:
        saleprice=None
    categories = [cat['id'] for cat in prod['categories']]
    info= {'id': pro_id,
                'product_name': name,
                'price':price,
                'saleprice':saleprice,
                'image_url': image_url,
                'categories': categories,
                'parent_category': 'dress'}
    need_scrape_info.append(info)

505844562
509214022
509466130
521942507
539186495
610843079
620122777
624108735
626648428
630248636
644679670
646364095
662187632
668379965
671596056
676811851
685181862
687071843
693621135
694385857
696066690
697620632
698248959
698395383
698585386
699698038
700654695
702498073
703333445
704734628
707056967
707472348
707485785
707961890
710067941
710561533
710939761
711815588
712327560
712568711
713641731
713837208
714223722
714534357
714534387
714540768
714543100
714656732
715079406
715235131
715374001
715586267
716572202
716737381
717401463
717413284
717588403
718074352
718310812
718815218
718858603
718987724
719148887
719954303
720666211
720667716
720699579
721381409
721387938
723015929
723386592
724023557
724255843
724615891
724641896
725166147
725274248
725278441
725520102
726416197
726498447
726570187
726570195
727159988
727175907
727325781
727336576
728219575
728219576
728884740
729508500
729626478
729789791
729794726
730186544
730196761
730783364
731552635
731576136
731739470


In [131]:
pickle.dump(need_scrape_info, open('extra_pants.pickle', 'wb'))

In [73]:
prod=requests.get(url).text

In [72]:
get_api_json('http://api.shopstyle.com/api/v2/products/459056803?pid=shopstyle')

AttributeError: 'str' object has no attribute 'loads'

In [48]:
json['categories']

[{'fullName': 'Dresses',
  'id': 'dresses',
  'localizedId': 'dresses',
  'name': 'Dresses',
  'numId': 26,
  'shortName': 'Dresses'}]

In [37]:
json_normalize(json).columns

Index(['alternateImages', 'badges', 'brand.id', 'brand.legacyUrlIdentifier',
       'brand.name', 'brand.urlIdentifier', 'brandedName', 'categories',
       'clickUrl', 'clusterUrl',
       ...
       'retailer.score', 'retailer.urlIdentifier', 'salePrice',
       'salePriceLabel', 'seeMoreLabel', 'seeMoreUrl', 'sizes', 'stock',
       'unbrandedName', 'urlIdentifier'],
      dtype='object', length=119)

In [ ]:
def parse_product(prod, parent_category):
        pro_id = prod['id']
        name = prod['name']
        image_url = prod['image']['sizes']['Original']['url']
        price=prod['priceLabel']
        try:
            saleprice=prod['salePriceLabel']
        except:
            saleprice=None
        categories = [cat['id'] for cat in prod['categories']]
        return {'id': pro_id,
                'product_name': name,
                'price':price,
                'saleprice':saleprice,
                'image_url': image_url,
                'categories': categories,
                'parent_category': parent_category}